In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import koreanize_matplotlib

In [4]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', 50) #None으로 설정하면 모든 행 출력

In [5]:
pd.options.display.float_format = '{:,.3f}'.format

In [6]:
import FinanceDataReader as fdr

In [7]:
fdr.StockListing('ETF/KR')

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1056985,2,95,0.010,"1,057,017.000",0.825,172270,182087,93049
1,357870,6,TIGER CD금리투자KIS(합성),55400,2,5,0.010,"55,395.000",0.810,92283,5112,69345
2,360750,4,TIGER 미국S&P500,21620,5,-55,-0.250,"21,600.000",17.067,2679260,57912,62784
3,069500,1,KODEX 200,33130,2,160,0.490,"33,163.000",-3.869,2392871,78862,56404
4,423160,6,KODEX KOFR금리액티브(합성),108935,2,25,0.020,"108,925.000",0.800,22584,2460,44412
...,...,...,...,...,...,...,...,...,...,...,...,...
925,407310,1,HANARO 200 TOP10,8005,2,20,0.250,"8,017.000",-3.271,568,4,16
926,227550,2,TIGER 200 산업재,6655,2,145,2.230,"6,663.000",-2.763,81,0,16
927,407300,2,HANARO Fn골프테마,5315,2,65,1.240,"5,334.000",-7.244,69,0,16
928,429990,7,SOL 한국형글로벌플랫폼&메타버스액티브,17015,2,90,0.530,"16,959.000",32.072,6,0,10


In [8]:
df_stocks = pd.read_csv('stocks.csv')

In [98]:
df_stocks.head(1)

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,005930,삼성전자,KOSPI,NaN,53900,1,500,0.940,54500,54500,53700,16788212,906959515200,321771279445000,5969782550,STK,통신 및 방송 장비 제조업,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조(...",1975-06-11,12월,한종희,http://www.samsung.com/sec,경기도


지금 내가 투자한 종목들의 성격에따라 투자자의 성향을 파악한뒤,(공격적인 투자인지, 보수적인 투자인지) -> 그에따른 종목추천을 하는방법을 찾기

1. 내가 투자한종목을 입력 받기
2. 내가 투자한 종목들의 변동률을 일정기간동안 평준화
   2-1 : 일정기간 - 단기투자, 중기투자, 장기투자에 따라 다르게
   2-2 : 단기투자 : 1개월 / 중기투자 : 4개월 / 장기투자 : 1년
4. 투자자의 성향파악 : 투자한 종목들의 변동률이 일정 이상이면 하이리스크 하이리턴형, 변동률이 낮으면 로우리스크 로우리턴형
5. 종목추천 : 변동률을 일정구간(4구간)으로 나눠서 투자자의 성향을 나눔
6. 각구간별로 데이터를 나눠놓음
7. 나눠놓은 데이터를 앞에서 분석한 성향에 따라 추천

In [18]:
df_stocks[df_stocks['Name'] == '삼성전자'][['Code','Name','Close']]

,Code,Name,Close
0,005930,삼성전자,53900


In [ ]:
fdr.DataReader('005930','2024-08')

In [37]:
samsung_df = fdr.DataReader('005930','2024-08')

In [ ]:
# 삼성전자의 변동률을 일정기간 동안 평준화
samsung_df['Change'].mean() * 100

In [75]:
df_stocks[df_stocks['Name'] == '삼성전자'][['Name','Code']]

,Name,Code
0,삼성전자,005930


In [77]:
df_stocks[df_stocks['Name'] == '삼성전자']['Code']

0    005930
Name: Code, dtype: object

In [83]:
# 투자자가 투자했던 종목
def my_invested_stocks(names) :
    return df_stocks[df_stocks['Name'] == name]['Code'].values

In [97]:
# 투자자가 투자했던 종목
def my_invested_stocks(names):
    # names가 리스트인지 확인하고, 리스트가 아닐 경우 리스트로 변환
    if not isinstance(names, list):
        names = [names]
        
    # 주어진 이름들에 해당하는 종목 코드 반환
    return df_stocks[df_stocks['Name'].isin(names)]['Code'].values

In [92]:
my_invested_stocks(['삼성전자','고려아연'])[0]

'005930'

In [ ]:
# 종목을 고변동, 중고변동, 중저변동, 저변동성으로 나누기
# 1개월, 4개월, 12개월단위로 3가지의 데이터 프레임이 필요
# 그 중에서 특정 분류기준에 따라 상위 몇개를 보여주는방식
# 시가총액, 규모 , 평균 거래량 등을 고려해볼 수 있을듯

In [ ]:
# 투자자의 성향을 파악
def classify_investor_type(codelist) : 
    코드리스트를 0부터 순회하면서 fdr.DataReader에 넣음
    기간은 기간별로 조회가능(1개월, 4개월, 1년) , 선택가능하게
    avg_change = fdr.DataReader('005930','2024-08')['Change'].mean()*100
    avg_change의 합을 종목수로 나눠서 투자자가 투자한 평균 변동률을 계산
    고변동 (예: +10% 이상)
    중고변동 (예: +5% ~ +10%)
    평균변동 (예: -5% ~ +5%)
    저변동 (예: -5% 이하)
    넷중 하나의 카테고리 반환


In [101]:
# 투자자의 성향을 파악
# codelist : 본인이 사전에 투자했던 종목, period : 분석 기간 (단기-1개월 중기-4개월 장기-12개월)
def classify_investor_type(codelist, period) :
    #특정 기간에 따르주식 데이터 수집
    avg_changes = []
    for code in codelist:
        data = fdr.DataReader(code, period)
        avg_change = data['Change'].mean() * 100 # 평균 변동률 계산
        avg_changes.append(avg_change)

    overall_avg_change = sum(avg_changes) / len(avg_changes)

    if overall_avg_change >= 10 :
        return 1
    elif overall_avg_change >= 5:
        return 2
    elif overall_avg_change >= -5:
        return 3
    else : 
        return 4

In [ ]:
# 성향을 기반으로 추천

def recommend_stocks(invester_type) : 
    recommendations = {
        # 목록을 편성할때, 분석기간의 30% 기간안에, 주가가 오른항목만 편성?
        # 혹은 내려가고 있어도 그게 저점일수 있으니까?
        # 이부분은 좀더 주식에 대한 학습이 필요할듯
        고변동 목록(invester_type == 1) : []
        중고변동 목록(invester_type == 2) : []
        중저변동 목록(invester_type == 3) : []
        저변동 목록(invester_type == 4) : []  

        # 여기에서 특정 Sector 별로 목록을 구분하는 함수가 필요
        # 변동성 목록중에 본인이 선호하는 Sector의 목록을 추릴수 있게

        # 정렬 기준은 시가총액/평균거래량/거래대금 등

    }

    # 최종적으로 본인의 투자성향에 맞는, 본인이 선호하는 섹터의 상위 목록을 반환
    # 그 반환된 목록의 plot과 정보를 시각화까지
    return recommendations.get(타입, 목록)


In [96]:
fdr.DataReader('005930','2024-08')['Change'].mean()*100

-0.47526723853894254

In [ ]:
def my_invested_stocks(name) :
    my_stocks = df_stocks[df_stocks['Name'] == name][['Name','Close']]
    
    
    